In [5]:
import os
import mxnet as mx
import pandas as pd
import cv2
import numpy as np

In [6]:
curpath = "E:/train/"
os.chdir(curpath)

In [7]:
img_lst = pd.read_csv('./model_train.lst', sep = '\t', header = None, names =('imgrank','type','path'))

In [8]:
img_lst.shape[0]

3022

In [9]:
def get_image(filename):
    img = cv2.imread(filename)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # change bgr to rgb
    img = cv2.resize(img, (224,224))
    img = np.swapaxes(img, 0, 2)
    img = np.swapaxes(img, 1, 2)
    img = img[np.newaxis, :]  # change to example,channel,width,height
    return img

In [10]:
# 提取中间层的输出进行训练模型；
def feature_extract(prefix, epoch):
    sym, arg_params, aux_params = mx.model.load_checkpoint(prefix, epoch)
    internals = sym.get_internals()
    fea_symbol = internals["fc1_output"]
    return(fea_symbol, arg_params, aux_params)

In [11]:
fea_model, arg_params, aux_params = feature_extract('D:/gitcode/DataScience/Python/python-mxnet-notebook/how_to/resnet-50',0)

In [12]:
# reset model
mod =mx.mod.Module(symbol=fea_model, context=mx.cpu())
mod.bind(for_training = False, data_shapes = [('data',(1,3,224,224))])  # should set the same with before
mod.set_params(arg_params, aux_params)

In [14]:
# define model 
from collections import namedtuple
Batch = namedtuple('Batch', ['data'])

In [16]:
len(img_lst)
i=1
curpath+img_lst['path'][i]
img = get_image(curpath+img_lst['path'][i])
img.shape

(1L, 3L, 224L, 224L)

In [17]:
# 获取训练结合的结果
result = np.zeros((len(img_lst), 1000))
for i in range(1,len(img_lst)):
    print(curpath+img_lst['path'][i])
    img = get_image(curpath+img_lst['path'][i])
    mod.forward(Batch([mx.nd.array(img)]))
    result[i,:] = mod.get_outputs()[0].asnumpy()

E:/train/BET\img_04931.jpg
E:/train/ALB\img_03904.jpg
E:/train/ALB\img_07441.jpg
E:/train/ALB\img_03304.jpg
E:/train/YFT\img_03340.jpg
E:/train/YFT\img_04082.jpg
E:/train/ALB\img_00660.jpg
E:/train/YFT\img_07250.jpg
E:/train/ALB\img_00834.jpg
E:/train/ALB\img_02185.jpg
E:/train/YFT\img_07569.jpg
E:/train/ALB\img_04878.jpg
E:/train/SHARK\img_01795.jpg
E:/train/ALB\img_02421.jpg
E:/train/YFT\img_02179.jpg
E:/train/ALB\img_06126.jpg
E:/train/YFT\img_01835.jpg
E:/train/ALB\img_05684.jpg
E:/train/ALB\img_05396.jpg
E:/train/DOL\img_03689.jpg
E:/train/ALB\img_05523.jpg
E:/train/ALB\img_07048.jpg
E:/train/BET\img_03347.jpg
E:/train/YFT\img_06481.jpg
E:/train/YFT\img_04628.jpg
E:/train/ALB\img_06629.jpg
E:/train/ALB\img_04112.jpg
E:/train/ALB\img_04798.jpg
E:/train/YFT\img_05708.jpg
E:/train/ALB\img_07335.jpg
E:/train/DOL\img_00770.jpg
E:/train/ALB\img_05040.jpg
E:/train/YFT\img_04759.jpg
E:/train/OTHER\img_05009.jpg
E:/train/ALB\img_01338.jpg
E:/train/ALB\img_03282.jpg
E:/train/ALB\img_03224.j

In [18]:
# train model ,x= result, y =img_lst['type]
# import sklearn 
# import xgboost as xgb

In [33]:
# 获取测试集合
img_test = pd.read_csv('./model_test.lst', sep = '\t', header = None, names =('imgrank','type','path'))
testresult = np.zeros((len(img_test),1000))
for i in range(1,len(img_test)):
    print("num_%i:%s" %(i,curpath+img_test['path'][i]))
    img = get_image(curpath+img_test['path'][i])
    mod.forward(Batch([mx.nd.array(img)]))
    testresult[i,:] = mod.get_outputs()[0].asnumpy()

num_1:E:/train/OTHER\img_05540.jpg
num_2:E:/train/ALB\img_02933.jpg
num_3:E:/train/OTHER\img_04391.jpg
num_4:E:/train/OTHER\img_03076.jpg
num_5:E:/train/ALB\img_04874.jpg
num_6:E:/train/YFT\img_01938.jpg
num_7:E:/train/SHARK\img_05519.jpg
num_8:E:/train/YFT\img_00641.jpg
num_9:E:/train/NoF\img_00949.jpg
num_10:E:/train/ALB\img_04778.jpg
num_11:E:/train/DOL\img_02267.jpg
num_12:E:/train/ALB\img_04154.jpg
num_13:E:/train/SHARK\img_06006.jpg
num_14:E:/train/NoF\img_05636.jpg
num_15:E:/train/ALB\img_03578.jpg
num_16:E:/train/NoF\img_03678.jpg
num_17:E:/train/OTHER\img_00261.jpg
num_18:E:/train/YFT\img_06092.jpg
num_19:E:/train/YFT\img_06417.jpg
num_20:E:/train/ALB\img_07701.jpg
num_21:E:/train/ALB\img_05358.jpg
num_22:E:/train/ALB\img_04446.jpg
num_23:E:/train/ALB\img_07257.jpg
num_24:E:/train/YFT\img_03198.jpg
num_25:E:/train/OTHER\img_00226.jpg
num_26:E:/train/SHARK\img_02944.jpg
num_27:E:/train/ALB\img_03038.jpg
num_28:E:/train/ALB\img_03927.jpg
num_29:E:/train/ALB\img_06619.jpg
num_30:

In [22]:
from sklearn import svm

In [35]:
x_train = result
y_train = img_lst['type']
x_test = testresult

In [29]:
clf = svm.SVC(C=10, kernel='rbf', degree=3, coef0=0.0, shrinking=True, probability=True, tol=0.001, cache_size=200, 
                  class_weight=None, verbose=False, max_iter=-1, random_state=None)

clf.fit(x_train, y_train)

C:\Anaconda2\lib\site-packages\sklearn\svm\base.py:85: DeprecationWarning: gamma=0.0 has been deprecated in favor of gamma='auto' as of 0.17. Backward compatibility for gamma=0.0 will be removed in 0.18
  DeprecationWarning)


SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.0, kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [84]:
# randomforest
from sklearn.ensemble import RandomForestClassifier
for i in range(10,300,10):
    rf = RandomForestClassifier(max_depth=5, n_estimators=i)
    rf.fit(x_train, y_train)
    ypred_rf = rf.predict(x_test)
    predict_temp = np.array(img_test['type']) - np.array(ypred_rf)
    count_num = np.count_nonzero(predict_temp)
    accuracy_rf = (len(predict_temp)-count_num)*1.0 / (len(predict_temp))
    print ('the accuracy of RandomForest in n_estimators  %i is %.4f' %(i,accuracy_rf))

the accuracy of RandomForest in n_estimators  10 is 0.5735
the accuracy of RandomForest in n_estimators  20 is 0.5669
the accuracy of RandomForest in n_estimators  30 is 0.5616
the accuracy of RandomForest in n_estimators  40 is 0.5695
the accuracy of RandomForest in n_estimators  50 is 0.5735
the accuracy of RandomForest in n_estimators  60 is 0.5735
the accuracy of RandomForest in n_estimators  70 is 0.5695
the accuracy of RandomForest in n_estimators  80 is 0.5695
the accuracy of RandomForest in n_estimators  90 is 0.5616
the accuracy of RandomForest in n_estimators  100 is 0.5669
the accuracy of RandomForest in n_estimators  110 is 0.5722
the accuracy of RandomForest in n_estimators  120 is 0.5775
the accuracy of RandomForest in n_estimators  130 is 0.5722
the accuracy of RandomForest in n_estimators  140 is 0.5722
the accuracy of RandomForest in n_estimators  150 is 0.5695
the accuracy of RandomForest in n_estimators  160 is 0.5815
the accuracy of RandomForest in n_estimators  170

In [85]:
# randomforest
from sklearn.ensemble import ExtraTreesClassifier
for i in range(10,300,10):
    forest = ExtraTreesClassifier(n_estimators=i)
    forest.fit(x_train, y_train)
    ypred_rf = forest.predict(x_test)
    predict_temp = np.array(img_test['type']) - np.array(ypred_rf)
    count_num = np.count_nonzero(predict_temp)
    accuracy_rf = (len(predict_temp)-count_num)*1.0 / (len(predict_temp))
    print ('the accuracy of RandomForest in n_estimators  %i is %.4f' %(i,accuracy_rf))

the accuracy of RandomForest in n_estimators  10 is 0.8146
the accuracy of RandomForest in n_estimators  20 is 0.8464
the accuracy of RandomForest in n_estimators  30 is 0.8503
the accuracy of RandomForest in n_estimators  40 is 0.8609
the accuracy of RandomForest in n_estimators  50 is 0.8556
the accuracy of RandomForest in n_estimators  60 is 0.8609
the accuracy of RandomForest in n_estimators  70 is 0.8742
the accuracy of RandomForest in n_estimators  80 is 0.8689
the accuracy of RandomForest in n_estimators  90 is 0.8596
the accuracy of RandomForest in n_estimators  100 is 0.8702
the accuracy of RandomForest in n_estimators  110 is 0.8662
the accuracy of RandomForest in n_estimators  120 is 0.8702
the accuracy of RandomForest in n_estimators  130 is 0.8715
the accuracy of RandomForest in n_estimators  140 is 0.8636
the accuracy of RandomForest in n_estimators  150 is 0.8715
the accuracy of RandomForest in n_estimators  160 is 0.8715
the accuracy of RandomForest in n_estimators  170

In [87]:
importance = forest.feature_importances_
importance

array([ 0.00094629,  0.00134317,  0.00089849,  0.00115406,  0.00088251,
        0.00109707,  0.000871  ,  0.00105949,  0.00098644,  0.00111496,
        0.00079228,  0.00114561,  0.00107619,  0.00118306,  0.00095656,
        0.00099969,  0.00186421,  0.00090495,  0.000664  ,  0.00099477,
        0.0009359 ,  0.00073982,  0.00102426,  0.00072552,  0.00079287,
        0.00104008,  0.00085387,  0.00092611,  0.00084696,  0.00099517,
        0.00114392,  0.00102947,  0.0008776 ,  0.00116076,  0.001428  ,
        0.00082029,  0.00086694,  0.00077872,  0.00083392,  0.00074907,
        0.00108551,  0.00072808,  0.00079135,  0.00113802,  0.00075863,
        0.00066509,  0.00093478,  0.00095513,  0.00088984,  0.0009311 ,
        0.00086213,  0.00094275,  0.00091673,  0.00104607,  0.00101779,
        0.00157701,  0.00106399,  0.00091287,  0.00099187,  0.00102359,
        0.00085415,  0.00078895,  0.00097526,  0.00073671,  0.00148588,
        0.00091675,  0.00087761,  0.00101583,  0.00093496,  0.00

In [ ]:
# randomforest
from sklearn.ensemble import GradientBoostingClassifier
for i in range(10,300,10):
    gbdt = GradientBoostingClassifier(learning_rate=0.05, subsample=0.7, max_depth=6, n_estimators=i)
    gbdt.fit(x_train, y_train)
    ypred_rf = gbdt.predict(x_test)
    predict_temp = np.array(img_test['type']) - np.array(ypred_rf)
    count_num = np.count_nonzero(predict_temp)
    accuracy_rf = (len(predict_temp)-count_num)*1.0 / (len(predict_temp))
    print ('the accuracy of RandomForest in n_estimators  %i is %.4f' %(i,accuracy_rf))

the accuracy of RandomForest in n_estimators  10 is 0.7523


In [65]:
ypred_rf[1:100]

array([ 0.,  0.,  0.,  0.,  0.,  7.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  7.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  7.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  7.,  0.,  4.,  4.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  4.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  7.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  4.,  7.,
        0.,  7.,  0.,  0.,  7.,  0.,  0.,  0.,  7.,  0.,  0.,  0.,  0.,
        7.,  0.,  7.,  0.,  0.,  0.,  0.,  0.])

In [73]:
from sklearn.metrics import confusion_matrix
y_pred = clf.predict(x_test)
predict_temp = np.array(img_test['type']) - np.array(y_pred)
count_num = np.count_nonzero(predict_temp)
accuracy_svm = (len(predict_temp)-count_num) / (len(predict_temp)/1.0)
proba_pred_svm = clf.predict_proba(x_test)
cm_svm = confusion_matrix(img_test['type'], y_pred)
print ('the accuracy of SVM is %.4f' %accuracy_svm)
print(cm_svm)

the accuracy of SVM is 0.9457
[[326   0   1   0   2   0   0   3]
 [  6  25   0   0   0   0   0   0]
 [  0   0  31   0   0   0   0   1]
 [  1   0   0  13   0   0   0   0]
 [ 18   0   0   0  85   0   0   0]
 [  2   0   0   0   0  54   0   0]
 [  2   0   0   0   0   0  37   0]
 [  4   0   0   0   1   0   0 143]]


In [74]:
y_pred[1:10]

array([ 5.,  0.,  5.,  5.,  0.,  7.,  6.,  7.,  4.])